# Demo collision detection 3D

In [ ]:
import torchlensmaker as tlm
import torch
import torch.nn
import math
from pprint import pprint

def make_random_rays(num_rays, start_x, end_x, max_y):
    rays_start = (torch.rand((num_rays, 3)) * 2 - 1) * max_y
    rays_start[:, 0] = start_x

    rays_end = (torch.rand((num_rays, 3)) * 2 - 1) * max_y
    rays_end[:, 0] = end_x

    rays_vectors = torch.nn.functional.normalize(rays_end - rays_start, dim=1)

    return torch.hstack((rays_start, rays_vectors))


test_rays = make_random_rays(
    num_rays=50,
    start_x=-15,
    end_x=50,
    max_y=20,
)

Tensor = torch.Tensor

def render_rays2D(P: Tensor, V: Tensor, length: float, color: str = "#ffa724") -> list[object]:
    rays_start = P
    rays_end = rays_start + length * V
    data = torch.hstack((rays_start, rays_end)).tolist()
    return {
        "type": "rays",
        "data": data,
        "color": color,
    }

def demo(rays):
    all_points = torch.empty((0, 3))
    all_normals = torch.empty((0, 3))
    P, V = test_rays[:, 0:2], test_rays[:, 2:4]

    groups = []
    groups.append(
        render_rays2D(P, V, 10.)
    )

    scene = {
        # groups
        "data": groups,
        "mode": "2D",
        "camera": "XY",
    }

    tlm.viewer.show(scene, ndigits=2)


demo(test_rays)